In [ ]:
from IPython.display import display, Javascript
import base64
import json
import asyncio

def setup_message_listener():
    """Setup listener for postMessage from parent window"""
    js_code = """
    new Promise((resolve) => {
        // Listen for messages from parent window
        window.addEventListener('message', function(event) {
            if (event.data.type === 'control-system-code') {
                // Store the code key in sessionStorage
                sessionStorage.setItem('controlSystemCodeKey', event.data.key);
                sessionStorage.setItem('controlSystemName', event.data.systemName);
                resolve(true);
            }
        });
        
        // If we already have a code key, use it immediately
        if (sessionStorage.getItem('controlSystemCodeKey')) {
            resolve(true);
        } else {
            // No message received yet, keep waiting
            setTimeout(() => resolve(false), 5000);
        }
    });
    """
    display(Javascript(js_code))

async def load_control_system():
    """Load control system code from localStorage using the provided key"""
    js_code = """
    new Promise((resolve) => {
        const codeKey = sessionStorage.getItem('controlSystemCodeKey');
        const systemName = sessionStorage.getItem('controlSystemName');
        
        if (!codeKey) {
            resolve({code: '', systemName: ''});
            return;
        }
        
        try {
            const base64Code = localStorage.getItem(codeKey);
            localStorage.removeItem(codeKey); // Clean up
            
            if (base64Code) {
                // Decode base64 to get the Python code
                const code = atob(base64Code);
                resolve({code: code, systemName: systemName});
            } else {
                resolve({code: '', systemName: ''});
            }
        } catch (error) {
            console.error('Error loading code:', error);
            resolve({code: '', systemName: ''});
        }
    });
    """
    
    result = await Javascript(js_code)
    return result

async def inject_code_into_notebook():
    """Main function to load and inject code into the notebook"""
    # Setup message listener first
    await setup_message_listener()
    
    # Load the control system code
    result = await load_control_system()
    code = result.get('code', '')
    system_name = result.get('systemName', 'Untitled System')
    
    if code:
        # Inject the code into the first code cell
        js_inject = f"""
        var code = {json.dumps(code)};
        var cells = Jupyter.notebook.get_cells();
        for (var i=0; i<cells.length; i++) {{
            if (cells[i].cell_type === 'code') {{
                cells[i].set_text(code);
                break;
            }}
        }}
        """
        display(Javascript(js_inject))
        print(f"Loaded control system: {system_name}")
    else:
        print("No control system code found. Ready for manual input.")

# Run the code injection
await inject_code_into_notebook()

# Default imports for control systems
import control
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
print("Notebook ready for simulation")